In [ ]:
!pip install ta
!pip install pandas-ta
!pip install investpy

In [ ]:
import pandas as pd
import numpy as np
import investpy
import seaborn as sns
from copy import copy
import matplotlib.pyplot as plt
import statistics as stats
import math
from ta.volume import MFIIndicator
from ta.volatility import AverageTrueRange
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import random
import multiprocessing as mp

# Import historique et mise en forme #

In [ ]:
df_source = investpy.get_etf_historical_data(etf='Lyxor UCITS NASDAQ-100 Daily Leverage',
                                            country='France',
                                            from_date='01/10/2006',
                                            to_date='31/12/2011')

df_source.drop(columns=['Currency','Exchange'], inplace=True)
df_source.reset_index(inplace=True)

In [ ]:
df_source_allTime = investpy.get_etf_historical_data(etf='Lyxor UCITS NASDAQ-100 Daily Leverage',
                                            country='France',
                                            from_date='01/10/2006',
                                            to_date='27/05/2022')

df_source_allTime.drop(columns=['Currency','Exchange'], inplace=True)
df_source_allTime.reset_index(inplace=True)

Vérification de la possible absence de lignes NaN

In [ ]:
df_source.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [ ]:
df_source_allTime.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

# Indicateur Alphatrend #
Version Python

In [ ]:
''' Code Pine script

coeff = input.float(1, 'Multiplier', step=0.1)
AP = input(14, 'Common Period')
ATR = ta.sma(ta.tr, AP)

src = input(close)

// Il s'agit du low & high de la bougie du jour 
upT = low - ATR * coeff
downT = high + ATR * coeff

AlphaTrend :=   
                // Cas MFI Uptrend 
                if (ta.mfi(hlc3, AP) >= 50 ) :
                    // Si le support Chandelier exit bougie du jour est en dessous Alphatrend précédent
                    // Alors Alphatrend ne bouge pas d'hier (flat)
                    // Si il est supérieur, l'Alphatrend monte à la nouvelle valeur
                    // Support dynamique
                    if(upT < AlphaTrend[1]): 
                        AlphaTrend[1]
                    else : 
                        upT
                
                // Cas MFI Downtrend
                else : 
                    // Si le support Chandelier exit bougie du jour est en dessus Alphatrend précédent
                    // Alors Alphatrend ne bouge pas d'hier (flat)
                    // Si il est inférieur, l'Alphatrend baisse à la nouvelle valeur
                    if(downT > AlphaTrend[1] ) : 
                        AlphaTrend[1] 
                    else : 
                        downT

color1 = 
    // Si supérieur à il a 2j -> Uptrend, vert
    if AlphaTrend > AlphaTrend[2] : 
        vert
    // Si inférieur à il a 2j -> Downtrend, rouge
    elif AlphaTrend < AlphaTrend[2] : 
        rouge 
    // Si = à il y a 2j, alors comparaison J-1 & J-3
    elif AlphaTrend[1] > AlphaTrend[3] :
        vert 
    else : 
        rouge

k1 = plot(AlphaTrend, color=color.new(bleu, 0), linewidth=3)
k2 = plot(AlphaTrend[2], color=color.new(rouge, 0), linewidth=3)

fill(k1, k2, color=color1)



// Buy si Alphatrend jour > Alphatrend J-2, croisement à la hausse (depuis Flat)
buySignalk = ta.crossover(AlphaTrend, AlphaTrend[2])
// Sell si Alphatrend jour < Alphatrend J-2, croisement à la hausse (depuis Flat)
sellSignalk = ta.crossunder(AlphaTrend, AlphaTrend[2])
'''

Remarques :
<li> Période ATR = Pérdiode MFI</li>
<li> Choix de MFI 50 & hlc3 pour tester ensuite validation trend</li>
<li> L'équivalent Supertrend se fait sur le haut et bas du jour, non de la période</li>

# Entrées Alphatrend + MFI #

## Fonctions ##

In [ ]:
def generate_alphatrend(df_in, mfi_seuil, mfi_seuil_entry, mfi_trigger, atr_l, m):
  '''Paramètres d'entrée : longueur MFI, longueur ATR, multiplier
  Retourne les colonnes Alphatrend, Alphatrend +2, signaux achat/vente
  :mfi_seuil = période MFI servant à délimiter up/down de l'alphatrend
  :mfi_trigger = période MFI pour recherche crossover, leading entry'''

  df = df_in.copy()

  # Colonnes MFI
  s_mfi = MFIIndicator(high=df.High, low=df.Low, close=df.Close, volume=df.Volume, window=mfi_seuil).money_flow_index()
  df["MFI_ref"] = s_mfi
  s_mfi = MFIIndicator(high=df.High, low=df.Low, close=df.Close, volume=df.Volume, window=mfi_trigger).money_flow_index()
  df["MFI_entry"] = s_mfi

  # Colonne ATR
  s_atr = AverageTrueRange(high=df.High, low=df.Low, close=df.Close, window=atr_l).average_true_range()
  df["ATR"] = s_atr

  # Lignes UpT et DownT
  df["UpT_limit"] = df["Low"] - df["ATR"] * m
  df["DownT_support"] = df["High"] + df["ATR"] * m

  # Suppression des lignes sans signal, en début de DataFrame
  df.dropna(inplace=True)
  df.reset_index(drop=True, inplace=True)

  # ===============================================
  # Calcul Alphatrend, en tant que série
  Alphatrend = [0]

  for i in range (1, df.shape[0]):
    # Cas Uptrend
    if df.at[i,"MFI_ref"] >= mfi_seuil_entry :
      if df.at[i,"UpT_limit"] < Alphatrend[-1] :
        # Flat
        Alphatrend.append(Alphatrend[-1])
      else :
        # Trailing stop loss Up
        Alphatrend.append(df.at[i,"UpT_limit"])
    
    # Cas Downtrend, MFI < 50
    else :
      if df.at[i,"DownT_support"] > Alphatrend[-1] :
        # Flat
        Alphatrend.append(Alphatrend[-1])
      else :
        # Trailing stop loss Down
        Alphatrend.append(df.at[i,"DownT_support"])
  # ===============================================
    # Ajout des lignes k1 et k2 en tant que colonnes
  if df.shape[0] == len(Alphatrend):
    df["Alphatrend_k1"] = Alphatrend
    # Ligne k2 décalée de 2j
    Alphatrend2 = df["Alphatrend_k1"].shift(periods=2, fill_value=0)
    df["Alphatrend_k2"] = Alphatrend2
  else :
    print("Erreur lors de la génération des lignes Alphatrend")

  # ===============================================
  # Génération des signaux d'achat/vente
  signal = [np.nan]

  for t in range (1, df.shape[0]):
    if (df.at[t,"Alphatrend_k1"] > df.at[t,"Alphatrend_k2"]) and (df.at[t-1,"Alphatrend_k1"] <= df.at[t-1,"Alphatrend_k2"]):
      signal.append("Buy")
    elif (df.at[t,"Alphatrend_k1"] < df.at[t,"Alphatrend_k2"]) and (df.at[t-1,"Alphatrend_k1"] >= df.at[t-1,"Alphatrend_k2"]):
      signal.append("Sell")
    else :
      signal.append(np.nan)
  
  # Signal shift index+1
  # Considérer l'ouverture pour les calculs de rendement
  #signal.pop()
  #signal.insert(0, np.nan)

  # Ajout des signaux en tant que colone
  if len(signal) == df.shape[0]:
    df["Signal"] = signal
  else :
    print("erreur lors de la génération des signaux achat / vente.")
  
  return df

In [ ]:
def MFI_crossover_indexes(df_in, entree):
  ''' Obtenir les index des dates auxquelles le MFI franchit la limite d'entrée vers le haut '''

  df=df_in.copy()

  # Ajout de la colonne MFI de la veille
  shift = df["MFI_entry"].shift(1)
  shift[0]=shift[1]
  df["MFI_lag"]=shift

  entries_list = df.index[ (df["MFI_entry"] > entree) & (df["MFI_lag"] <= entree) ].to_list()

  # Ajout de 1 à l'index pour avoir la date du lendemain où le crossover a eu lieu
  #entries_list = [x +1 for x in entries_list]

  return entries_list

## Identification des entrées ##

In [ ]:
# Genère le df contenant Alphatrend et ses sous-jacents
# Rappel : ordre des paramètres 
# df_in, mfi_seuil, mfi_seuil_entry, mfi_trigger, atr_l, m
df_TA = generate_alphatrend(df_source, 14, 50, 17, 14, 1)

In [ ]:
# Arrondis
cols_to_round = df_TA.columns[6:13].to_list()

for c in cols_to_round:
  df_TA[c] = df_TA[c].round(2)

In [ ]:
df_TA

,Date,Open,High,Low,Close,Volume,MFI_ref,MFI_entry,ATR,UpT_limit,DownT_support,Alphatrend_k1,Alphatrend_k2,Signal
0,2006-10-24,19.06,19.08,18.88,18.90,664,57.34,57.18,0.36,18.52,19.44,0.00,0.00,NaN
1,2006-10-25,18.84,18.84,18.81,18.81,164,54.60,56.69,0.34,18.47,19.18,18.47,0.00,Buy
2,2006-10-26,19.06,19.12,18.88,18.92,6289,64.43,67.51,0.34,18.54,19.46,18.54,0.00,NaN
3,2006-10-27,19.04,19.23,19.00,19.04,1750,64.59,69.61,0.34,18.66,19.57,18.66,18.47,NaN
4,2006-10-30,18.68,18.78,18.55,18.78,697,61.08,66.77,0.35,18.20,19.13,18.66,18.54,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,2011-12-23,23.61,23.70,23.48,23.70,17797,16.57,27.95,0.68,22.80,24.38,23.48,23.48,NaN
1315,2011-12-27,24.00,24.02,23.69,23.91,8386,20.32,28.11,0.66,23.03,24.68,23.48,23.48,NaN
1316,2011-12-28,23.81,23.97,23.75,23.75,43541,18.72,20.04,0.63,23.12,24.60,23.48,23.48,NaN
1317,2011-12-29,23.70,23.84,23.57,23.84,3901,15.66,16.39,0.60,22.97,24.44,23.48,23.48,NaN


### Système de double Trigger ###
Leading : +10j après cross over MFI17 @38<br />
Lagging : si signal Buy dans période Leading +10 -> Achat

In [ ]:
entries_indexes = MFI_crossover_indexes(df_TA, 38)

In [ ]:
print(entries_indexes)

[35, 55, 85, 89, 94, 99, 105, 114, 170, 217, 219, 248, 255, 258, 267, 270, 282, 318, 325, 336, 340, 348, 383, 404, 413, 462, 471, 475, 506, 535, 597, 628, 650, 679, 766, 790, 795, 835, 967, 1128, 1130, 1142, 1187, 1233, 1241, 1297, 1299]


In [ ]:
df_mfi = df_TA.filter(items = entries_indexes, axis=0)
df_mfi = df_mfi[['Date','Open']]
df_mfi.reset_index(drop=True, inplace=True)

In [ ]:
# Trigger activé dès le jour où cloture valide le crossover au dessus du seuil défini
df_TA["MFI_trigger"] = np.nan

for ei in entries_indexes:
  df_TA.at[ei, "MFI_trigger"] = 1

In [ ]:
# Glisse le trigger sur les x prochains jours
df_TA["MFI_trigger"] = df_TA["MFI_trigger"].interpolate(method='pad', limit=9)

In [ ]:
# Index des positions acheteuses
buy_list = df_TA.index[ (df_TA["Signal"] == "Buy") & (df_TA["MFI_trigger"] == 1) ].to_list()

In [ ]:
print(buy_list)

[56, 86, 105, 115, 120, 170, 179, 217, 283, 356, 383, 390, 512, 599, 602, 605, 629, 652, 686, 771, 792, 798, 836, 844, 1143, 1150, 1187, 1238, 1242, 1299]


In [ ]:
# On considère l'ouverture du jour où les 2 signaux d'achats valident l'entrée
buy_list = [b+1 for b in buy_list]
print(buy_list)

[57, 87, 106, 116, 121, 171, 180, 218, 284, 357, 384, 391, 513, 600, 603, 606, 630, 653, 687, 772, 793, 799, 837, 845, 1144, 1151, 1188, 1239, 1243, 1300]


In [ ]:
df_entries = df_TA.filter(items = buy_list, axis=0)
df_entries = df_entries[['Date','Open']]
df_entries.reset_index(drop=True, inplace=True)
df_entries

,Date,Open
0,2007-01-16,20.66
1,2007-02-27,19.63
2,2007-03-26,18.78
3,2007-04-11,18.87
4,2007-04-18,19.05
5,2007-06-29,21.26
6,2007-07-13,22.43
7,2007-09-06,22.02
8,2007-12-07,22.45
9,2008-03-26,14.84


### Calcul identique pour période all time ###

In [ ]:
# DataFrame avec indicateurs techniques Alphtarend & MFI leading
df_TA_allTime = generate_alphatrend(df_source_allTime, 14, 50, 17, 14, 1)

# Arrondis
cols_to_round = df_TA_allTime.columns[6:13].to_list()
for c in cols_to_round:
  df_TA_allTime[c] = df_TA_allTime[c].round(2)

# Identification des Trigger MFI leading
entries_indexes = MFI_crossover_indexes(df_TA_allTime, 38)

df_TA_allTime["MFI_trigger"] = np.nan
for ei in entries_indexes:
  df_TA_allTime.at[ei, "MFI_trigger"] = 1

df_TA_allTime["MFI_trigger"] = df_TA_allTime["MFI_trigger"].interpolate(method='pad', limit=9)

# Index des positions acheteuses (puis entrée à J+1)
buy_list_allTime = df_TA_allTime.index[ (df_TA_allTime["Signal"] == "Buy") & (df_TA_allTime["MFI_trigger"] == 1) ].to_list()
buy_list_allTime = [b+1 for b in buy_list_allTime]

## Analyse stat des entrées ##
Performances à sortie fixe

### Fonctions ###

In [ ]:
def fix_oob_index(a, limite):
  ''' Necessaire pour corriger si l'entrée est trop proche de notre date limite max.
  Doit préceder generate_future_dates en conséquence'''
  for element in range(0,len(a)):
    if a[element] > limite :
      a[element] = limite
  return a

In [ ]:
def generate_future_dates(df, entries_list):
  '''Générer les dates futures après identification des entrées
  Puis calcul des variations'''

  arr_entries = np.asarray(entries_list)

  arr_e2s = arr_entries + 10
  arr_e3s = arr_entries + 15
  arr_e1m = arr_entries + 21
  arr_e3m = arr_entries + 63
  
  # Aggregation des matrices sous forme de liste en vue de traitements en boucle
  dates_ech = [ arr_e2s, arr_e3s, arr_e1m, arr_e3m]

  # On fixe le plus grand index possible, au cas où la projection 10j, 15j ou 1 mois en avant soit out of bounds
  date_limite = df.shape[0]-1

  # Correction des éventuelles dates futures out of bounds, selon fonction définie précedemment
  for echeance in dates_ech:
    echeance = fix_oob_index(echeance,date_limite)

  # Conversion des index de array -> liste
  # Sélection du Close à la date correspondant à l'index
  s_3s = df.loc[list(arr_e3s),'Close'].to_list()
  s_2s = df.loc[list(arr_e2s),'Close'].to_list()
  s_1m = df.loc[list(arr_e1m),'Close'].to_list()
  s_3m = df.loc[list(arr_e3m),'Close'].to_list()

  # Création d'un dataframe d'entrée selon les index indentifiés
  df_res = df.filter(items = entries_list, axis=0)

  # Ajout des séries comme nouvelles colonnes
  df_res['Close_2s'] = s_2s
  df_res['Close_3s'] = s_3s
  df_res['Close_1m'] = s_1m
  df_res['Close_3m'] = s_3m

  # Performance pure
  df_res['var_2s'] = round( (df_res['Close_2s']-df_res['Close'])/df_res['Close'] ,2)
  df_res['var_3s'] = round( (df_res['Close_3s']-df_res['Close'])/df_res['Close'] ,2)
  df_res['var_1m'] = round( (df_res['Close_1m']-df_res['Close'])/df_res['Close'] ,2)
  df_res['var_3m'] = round( (df_res['Close_3m']-df_res['Close'])/df_res['Close'] ,2)

  # Performance ajustée des frais d'achat vente. 1.1% en moyenne haut
  df_res['var_2s_adj'] = round( (df_res['Close_2s']-df_res['Close'])/df_res['Close'] -0.011,2)
  df_res['var_3s_adj'] = round( (df_res['Close_3s']-df_res['Close'])/df_res['Close'] -0.011,2)
  df_res['var_1m_adj'] = round( (df_res['Close_1m']-df_res['Close'])/df_res['Close'] -0.011,2)
  df_res['var_3m_adj'] = round( (df_res['Close_3m']-df_res['Close'])/df_res['Close'] -0.011,2)

  # Suppression de la dernière ligne, car n'ayant pas abouti dans le temps, va fausser les résultats d'analyse
  df_res.drop(df_res.tail(1).index, inplace = True)

  # On ne conserve que les champs liés aux performances à échéance
  df_res = df_res[['Date','Close','var_2s','var_3s','var_1m','var_3m','var_2s_adj','var_3s_adj','var_1m_adj','var_3m_adj']]
  df_res.reset_index(drop=True, inplace=True)

  return df_res

In [ ]:
def kill_multiple_entries(df, echeance_fixe):
  ''' Dédoublonnage itératif, adapté à sortie fixe + x jours'''

  try:
      j=1
      last_row = df.shape[0]-1

      # L'index tend à se réduire au fur et à mesure des suppressions de lignes
      while j <= last_row:
        
        # Tant que la ligne suivante n'est pas espacée de 21j, on la supprime, reset l'index et reverifie
        while ((df["Date"][j] - df["Date"][j-1]).days < echeance_fixe):
          df.drop(index=j, inplace=True)
          df.reset_index(drop=True, inplace=True)
        
        # Passage à la ligne suivante. Variable mise à jour par la nouvelle taille du tableau
        last_row = df.shape[0]-1
        j+=1

    # Gestion de l'exception quand la dernière ligne est supprimée    
  except KeyError:
      pass

In [ ]:
def calc_performance(df, echeance):
  ''' Calculer la performance cumulée en % à 1 mois '''

  # Calcul performance investissement initial
  perf_list=[1000]

  # Selection de la dernière colonne, variation à 1 mois
  perfs = df[echeance].to_list()

  # Investissement de base, 1000
  invest = 1000

  # Calcul de la performance cumulée
  for i in range (0, df.shape[0]):
    invest = invest * (1 + perfs[i])
    perf_list.append(invest)

  # Equivalence en %
  perfP = round( ((perf_list[-1]-perf_list[0])/perf_list[0])*100 ,2)

  return perfP

In [ ]:
def calc_WinLossRatio(df, echeance):
  ''' Obtention du nombre de trades positifs à 2 semaines (10j)'''
  compte = df.apply(lambda x : 
                          True if x[echeance] > 0 
                          else False, axis = 1)
  # Count number of True in the series
  wr_ratio = round( len(compte[compte == True].index) / df.shape[0], 2)
  return wr_ratio

In [ ]:
def calc_profit_factors(df, echeance, WR):
  ''' Donne le profit factor classique et alternatif, sur la base du dataset df_perf et du Win Loss Ratio obtenu
  Profit factor classique = (Gross winning trades) / (Gross losing trades)
  Alternative = (Win Rate * Avg Win) / (Loss Rate * Avg Loss)'''
  pf=[]
  
  df_perf_pos = df.loc[ (df[echeance] > 0) ]
  df_perf_pos["gain"] = 1000 * df_perf_pos[echeance] 
  gross_gains = df_perf_pos["gain"].sum()

  df_perf_neg = df.loc[ (df[echeance] <= 0) ]
  df_perf_neg["loss"] = 1000 * df_perf_neg[echeance] 
  gross_losses = abs(df_perf_neg["loss"].sum())

  gross_profit_factor = round(gross_gains / gross_losses, 2)
  pf.append(gross_profit_factor)

  pfa_pos = df_perf_pos[echeance].mean() * WR
  pfa_neg = ( abs(df_perf_neg[echeance]).mean() ) * (1-WR)
  pfa = round(pfa_pos / pfa_neg, 2)
  pf.append(pfa)

  return pf

### Résultat et visualisation, entrées fixes ###

In [ ]:
bh = ((df_source['Close'].iloc[-1] - df_source['Close'].iloc[0])/df_source['Close'].iloc[0])*100
print("Buy & Hold : {:.2f} %".format(bh))

Buy & Hold : 37.27 %


In [ ]:
bh_allTime = ((df_source_allTime['Close'].iloc[-1] - df_source_allTime['Close'].iloc[0])/df_source_allTime['Close'].iloc[0])*100
print("Buy & Hold : {:.2f} %".format(bh_allTime))

Buy & Hold : 3114.45 %


In [ ]:
# Ajout des performances à +10, +15 et +21j
df_dates = generate_future_dates(df_TA, buy_list)
df_dates.shape

(29, 10)

In [ ]:
# Suppression des entrées qui se chevauchent sur 21j
kill_multiple_entries(df_dates, 10)
df_dates.shape

(23, 10)

In [ ]:
df_dates

,Date,Close,var_2s,var_3s,var_1m,var_3m,var_2s_adj,var_3s_adj,var_1m_adj,var_3m_adj
0,2007-01-16,20.56,-0.07,-0.06,-0.05,-0.07,-0.08,-0.08,-0.06,-0.08
1,2007-02-27,19.18,-0.06,-0.06,-0.04,0.10,-0.07,-0.07,-0.05,0.09
2,2007-03-26,18.60,0.01,0.02,0.08,0.10,-0.00,0.01,0.07,0.09
3,2007-04-11,18.73,0.05,0.08,0.08,0.19,0.04,0.07,0.07,0.18
4,2007-06-29,21.16,0.08,0.08,0.02,0.08,0.07,0.07,0.01,0.07
5,2007-07-13,22.56,-0.04,-0.08,-0.07,0.10,-0.05,-0.09,-0.08,0.09
6,2007-09-06,21.96,-0.00,0.05,0.09,-0.04,-0.01,0.04,0.08,-0.05
7,2007-12-07,22.49,-0.01,-0.06,-0.19,-0.41,-0.02,-0.08,-0.20,-0.42
8,2008-03-26,14.83,0.01,-0.00,0.07,0.12,-0.00,-0.01,0.06,0.11
9,2008-05-05,17.36,0.06,-0.03,0.06,-0.14,0.04,-0.04,0.04,-0.15


In [ ]:
# Opération identique sur la période all time
df_dates_allTime = generate_future_dates(df_TA_allTime, buy_list_allTime)
kill_multiple_entries(df_dates_allTime, 10)
df_dates_allTime.shape

(58, 10)

In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=df_dates['var_3m_adj'], name="+3 mois, 2006-2011", quartilemethod="inclusive", marker_color='seagreen'))
fig.add_trace(go.Box(x=df_dates_allTime['var_3m_adj'], name="+3 mois, 2006-jour", quartilemethod="inclusive", marker_color='seagreen'))

fig.add_trace(go.Box(x=df_dates['var_1m_adj'], name="+21j, 2006-2011", quartilemethod="inclusive", marker_color='lightseagreen'))
fig.add_trace(go.Box(x=df_dates_allTime['var_1m_adj'], name="+21j, 2006-jour", quartilemethod="inclusive", marker_color='lightseagreen'))

fig.add_trace(go.Box(x=df_dates['var_3s_adj'], name="+15j, 2006-2011", quartilemethod="inclusive", marker_color='LightSkyBlue'))
fig.add_trace(go.Box(x=df_dates_allTime['var_3s_adj'], name="+15j, 2006-jour", quartilemethod="inclusive", marker_color='LightSkyBlue'))

fig.add_trace(go.Box(x=df_dates['var_2s_adj'], name="+10j, 2006-2011", quartilemethod="inclusive", marker_color='indianred'))
fig.add_trace(go.Box(x=df_dates_allTime['var_2s_adj'], name="+10j, 2006-jour", quartilemethod="inclusive", marker_color='indianred'))
fig.show()

### Performance & Win Rate "Turtle"
sans stop loss, et à date de sortie fixe 3 semaines

In [ ]:
perf_turtle_allTime = calc_performance(df_dates_allTime, "var_3s_adj")
print("Performance Turtle all time {:.2f} %".format(perf_turtle_allTime))
print("Buy & Hold  all time {:.2f} %".format(bh_allTime))

Performance Turle all time 44.59 %
Buy & Hold  all time 3114.45 %


In [ ]:
WR_turtle_allTime = calc_WinLossRatio(df_dates_allTime, "var_3s_adj")
print("Win Rate Turtle all time", WR_turtle_allTime)

Win Rate Turtle all time 0.52


In [ ]:
PF_turtle = calc_profit_factors(df_dates_allTime, "var_3s_adj", WR_turtle_allTime)
print("Profit Factors Turtle all time 3 semaines  : {:.2f} & {:.2f}". format(PF_turtle[0],PF_turtle[1]) )

Profit Factors Turtle all time : 1.39 & 2.51


Turtle à 3 mois

In [ ]:
perf_turtle_allTime = calc_performance(df_dates_allTime, "var_3m_adj")
print("Performance Turtle all time {:.2f} %".format(perf_turtle_allTime))
print("Buy & Hold  all time {:.2f} %".format(bh_allTime))

Performance Turle all time 4882.56 %
Buy & Hold  all time 3114.45 %


Le turtle à 60j sans stop loss gagne le Buy & Hold !!!

In [ ]:
WR_turtle_allTime = calc_WinLossRatio(df_dates_allTime, "var_3m_adj")
print("Win Rate Turtle all time", WR_turtle_allTime)

Win Rate Turtle all time 0.66


In [ ]:
PF_turtle = calc_profit_factors(df_dates_allTime, "var_3m_adj", WR_turtle_allTime)
print("Profit Factors Turtle all time 3 mois : {:.2f} & {:.2f}". format(PF_turtle[0],PF_turtle[1]) )

Profit Factors Turtle all time : 1.60 & 1.63


In [ ]:
df_dates_allTime

,Date,Close,var_2s,var_3s,var_1m,var_3m,var_2s_adj,var_3s_adj,var_1m_adj,var_3m_adj
0,2007-01-16,20.56,-0.07,-0.06,-0.05,-0.07,-0.08,-0.08,-0.06,-0.08
1,2007-02-27,19.18,-0.06,-0.06,-0.04,0.10,-0.07,-0.07,-0.05,0.09
2,2007-03-26,18.60,0.01,0.02,0.08,0.10,-0.00,0.01,0.07,0.09
3,2007-04-11,18.73,0.05,0.08,0.08,0.19,0.04,0.07,0.07,0.18
4,2007-06-29,21.16,0.08,0.08,0.02,0.08,0.07,0.07,0.01,0.07
5,2007-07-13,22.56,-0.04,-0.08,-0.07,0.10,-0.05,-0.09,-0.08,0.09
6,2007-09-06,21.96,-0.00,0.05,0.09,-0.04,-0.01,0.04,0.08,-0.05
7,2007-12-07,22.49,-0.01,-0.06,-0.19,-0.41,-0.02,-0.08,-0.20,-0.42
8,2008-03-26,14.83,0.01,-0.00,0.07,0.12,-0.00,-0.01,0.06,0.11
9,2008-05-05,17.36,0.06,-0.03,0.06,-0.14,0.04,-0.04,0.04,-0.15


# Identification des sorties #

## Fonctions ##

In [ ]:
def last_3_down(df, row):
  ''' Vérification de si les 3 dernières clotures ont été sous les k1 et k2 
  Dans un mouvement haussier, k1 tombe toujours avant k2
  Un signal sous k1 est donc nécessairement sous k2'''
  if row <= 2:
    return False
  else:
    if(
        (df.at[row,'Close'] <= df.at[row,'Alphatrend_k1']) and
        (df.at[row-1,'Close'] <= df.at[row-1,'Alphatrend_k1']) and
        (df.at[row-2,'Close'] <= df.at[row-2,'Alphatrend_k1'])
        ):
      return True
    else:
      return False

In [ ]:
def get_exits_list(df, el, sl):
  ''' Sortie dans les cas suivants :
              - Signal Alphatrend passé à Sell
              - 3 bouges cloturées en dessous des lignes k1 et k2
              - Stop loss atteint à la cloture
      On utilisera le trailing stop Alphatrend plutôt qu'un objectif fixe de sortie'''
  
  exits_list=[]
  
  for e in range (0, len(el)):
    # On isole l'index de la ligne d'entrée
    row_index = el[e]
    # Pointeur commence les vérifications à partir de la ligne suivante
    r = row_index+1
    # Indicateur qu'une limite a été trouvée ou la fin des données atteinte
    touch = False

    # Définition des limites
    stop_loss = df.at[row_index,'Close'] * sl
    
    # Test l'une des limites est atteinte à la cloture
    while(touch == False):  
      if( r >= df.shape[0]-1 ):
        exits_list.append(df.shape[0]-1)
        touch = True
      elif( 
            (df.at[r,'Close'] <= stop_loss) or
            (df.at[r,'Signal'] == 'Sell') or
            (last_3_down(df, r))
           ):
        exits_list.append(r+1)
        touch = True
      else:
        r+=1
    e+=1

  return exits_list

In [ ]:
def generate_df_perf(df_indicateurs, le, ls):
  # Préparation au merge des entrées et sorties dans un seul et même DataFrame
  df_entries = df_indicateurs.filter(items = le, axis=0)
  df_entries = df_entries[['Date','Open']]
  df_entries.reset_index(drop=True, inplace=True)

  df_exits = df_indicateurs.filter(items = ls, axis=0)
  df_exits = df_exits[['Date','Open']]
  df_exits.reset_index(drop=True, inplace=True)

  # Jointure sur index réinitialisé
  df_P = df_entries.join(df_exits, lsuffix='_entry', rsuffix='_exit')
  df_P = df_P[['Date_entry','Date_exit','Open_entry','Open_exit']]
  # Variation positions Long
  df_P['Perf']= round( (df_P['Open_exit']-df_P['Open_entry'])/df_P['Open_entry'], 2)

  
  # Dédoublonnage itératif
  iteration = 1
  
  while iteration <4:
    
    doublons = []
    j=1
    # Obtention des dates de sorties, moins la dernière
    s_exit_dates = df_P["Date_exit"]
    s_exit_dates.drop(s_exit_dates.tail(1).index, inplace=True)
    
    # Obtention des index où les entrées chevauchent à une même date de sortie
    for date_sortie in s_exit_dates:
      if date_sortie > df_P["Date_entry"][j]:
        doublons.append(j)
      j+=1
    
    # Suppression des index identifiés comme doublons d'entrées
    df_P.drop(index=doublons, inplace=True)
    df_P.reset_index(drop=True, inplace=True)
    
    # Fin n-ième intération
    iteration+=1

  return df_P

## Performance avec sorties stratégie ##

In [ ]:
# Liste d'entrées brutes en entrées, et se recoupant potentiellement avec sorties
# Stop loss exprimé en %
exits_lists_allTime = get_exits_list(df_TA_allTime, buy_list_allTime, 0.9)

In [ ]:
# Contrôle que chaque entrée dispose bien de sa sortie
if len(exits_lists_allTime) == len(buy_list_allTime):
  df_Perf_allTime = generate_df_perf(df_TA_allTime, buy_list_allTime, exits_lists_allTime)

In [ ]:
df_Perf_allTime['adj_Perf'] = round(df_Perf_allTime['Perf'] - 0.011, 2)

In [ ]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=df_Perf_allTime['adj_Perf'], name="Perfs all Time", quartilemethod="inclusive", marker_color='lightseagreen'))
fig.show()

In [ ]:
perf_allTime = calc_performance(df_Perf_allTime, "adj_Perf")
print("Performance all time {:.2f} %".format(perf_allTime))
print("Buy & Hold  all time {:.2f} %".format(bh_allTime))

Performance all time 46.12 %
Buy & Hold  all time 3114.45 %


In [ ]:
WR_allTime = calc_WinLossRatio(df_Perf_allTime, "adj_Perf")
print("Win Rate all time", WR_allTime)

Win Rate all time 0.39


In [ ]:
df_Perf_allTime

,Date_entry,Date_exit,Open_entry,Open_exit,Perf,adj_Perf
0,2007-01-16,2007-01-23,20.66,19.71,-0.05,-0.06
1,2007-02-27,2007-03-01,19.63,18.31,-0.07,-0.08
2,2007-03-26,2007-04-04,18.78,18.36,-0.02,-0.03
3,2007-04-11,2007-07-27,18.87,21.61,0.15,0.14
4,2007-09-06,2007-09-12,22.02,21.29,-0.03,-0.04
5,2007-12-07,2007-12-18,22.45,20.95,-0.07,-0.08
6,2008-03-26,2008-04-17,14.84,15.09,0.02,0.01
7,2008-05-05,2008-05-23,17.55,16.85,-0.04,-0.05
8,2008-11-05,2008-11-07,10.43,9.01,-0.14,-0.15
9,2009-03-16,2009-05-13,7.35,9.53,0.30,0.29


#### Améliorations possibles ? ####
Entrées ratés post COVID pour MFI crossover trop loin, ou rebond reprise Buy sans que MFI repasse sous 38<br/>
Fin 2019 une sortie coupée par la règle des 3 bougies qui semble faire sortir trop tôt<br />
<p>
<li>Tester Alphatrend sur Nasdaq 100 (sans MFI ne semble pas faire de différence) + Donchian Channel 10 pour valider l'entrée (HH + HL).</li>
<li>Ceci étant les entrées n'étaient pas si mauvaises en Turtle : trouver meilleure échéance de sortie ?</li>
<li>Ou alors tester avec MFI 17 exit 87</li>
<li> Ou trouver meilleure combinaison MFI de sortie ?</li>

# Zone de conception #

## Création de la fonction Alphatrend ##
lignes k1, k2, signaux d'achat et vente

In [ ]:
df_ta = df_source.copy()

In [ ]:
s_mfi = MFIIndicator(high=df_source.High, low=df_source.Low, close=df_source.Close, volume=df_source.Volume, window=14).money_flow_index()
df_ta["MFI_14"] = s_mfi

In [ ]:
s_atr = AverageTrueRange(high=df_source.High, low=df_source.Low, close=df_source.Close, window=14).average_true_range()
df_ta["ATR_14"] = s_atr

In [ ]:
multiplier = 1.0
df_ta["UpT_limit"] = df_ta["Low"] - df_ta["ATR_14"] * multiplier
df_ta["DownT_support"] = df_ta["High"] + df_ta["ATR_14"] * multiplier

In [ ]:
origine = df_ta.shape[0]

In [ ]:
df_ta.dropna(inplace=True)
correction = df_ta.shape[0]
print("{} lignes supprimées".format(origine-correction))

13 lignes supprimées


In [ ]:
df_ta.reset_index(drop=True, inplace=True)
df_ta.head(1)

,Date,Open,High,Low,Close,Volume,MFI_14,ATR_14,UpT_limit,DownT_support
0,2006-10-19,18.58,18.58,18.55,18.55,1069,53.531988,0.340714,18.209286,18.920714


In [ ]:
Alphatrend = [0]

for i in range (1, df_ta.shape[0]):
  # Cas Uptrend
  if df_ta.at[i,"MFI_14"] >= 50 :
     if df_ta.at[i,"UpT_limit"] < Alphatrend[-1] :
       # Flat
       Alphatrend.append(Alphatrend[-1])
     else :
       # Trailing stop loss Up
       Alphatrend.append(df_ta.at[i,"UpT_limit"])
  
  # Cas Downtrend, MFI < 50
  else :
    if df_ta.at[i,"DownT_support"] > Alphatrend[-1] :
      # Flat
      Alphatrend.append(Alphatrend[-1])
    else :
      # Trailing stop loss Down
      Alphatrend.append(df_ta.at[i,"DownT_support"])

In [ ]:
df_ta.shape[0] == len(Alphatrend)

True

In [ ]:
df_ta["Alphatrend"] = Alphatrend

In [ ]:
# Ligne k2 décalée de 2j
Alphatrend2 = df_ta["Alphatrend"].shift(periods=2, fill_value=0)

In [ ]:
df_ta["Alphatrend_l2"] = Alphatrend2

In [ ]:
df_ta.head()

,Date,Open,High,Low,Close,Volume,MFI_14,ATR_14,UpT_limit,DownT_support,Alphatrend,Alphatrend_l2
0,2006-10-19,18.58,18.58,18.55,18.55,1069,53.531988,0.340714,18.209286,18.920714,0.000000,0.000000
1,2006-10-20,18.65,18.75,18.42,18.55,1750,57.990656,0.339949,18.080051,19.089949,18.080051,0.000000
2,2006-10-23,18.76,19.28,18.62,19.19,544,59.412343,0.367810,18.252190,19.647810,18.252190,0.000000
3,2006-10-24,19.06,19.08,18.88,18.90,664,57.335023,0.363680,18.516320,19.443680,18.516320,18.080051
4,2006-10-25,18.84,18.84,18.81,18.81,164,54.603703,0.344132,18.465868,19.184132,18.516320,18.252190


In [ ]:
signal = [np.nan]

for t in range (1, df_ta.shape[0]):
  if (df_ta.at[t,"Alphatrend"] > df_ta.at[t,"Alphatrend_l2"]) and (df_ta.at[t-1,"Alphatrend"] <= df_ta.at[t-1,"Alphatrend_l2"]):
    signal.append("Buy")
  elif (df_ta.at[t,"Alphatrend"] < df_ta.at[t,"Alphatrend_l2"]) and (df_ta.at[t-1,"Alphatrend"] >= df_ta.at[t-1,"Alphatrend_l2"]):
    signal.append("Sell")
  else :
    signal.append(np.nan)

In [ ]:
len(signal) == df_ta.shape[0]

True

In [ ]:
df_ta["Signal"] = signal

In [ ]:
df_ta.head()

,Date,Open,High,Low,Close,Volume,MFI_14,ATR_14,UpT_limit,DownT_support,Alphatrend,Alphatrend_l2,Signal
0,2006-10-19,18.58,18.58,18.55,18.55,1069,53.531988,0.340714,18.209286,18.920714,0.000000,0.000000,NaN
1,2006-10-20,18.65,18.75,18.42,18.55,1750,57.990656,0.339949,18.080051,19.089949,18.080051,0.000000,Buy
2,2006-10-23,18.76,19.28,18.62,19.19,544,59.412343,0.367810,18.252190,19.647810,18.252190,0.000000,NaN
3,2006-10-24,19.06,19.08,18.88,18.90,664,57.335023,0.363680,18.516320,19.443680,18.516320,18.080051,NaN
4,2006-10-25,18.84,18.84,18.81,18.81,164,54.603703,0.344132,18.465868,19.184132,18.516320,18.252190,NaN


In [ ]:
print(pd.__version__)

1.3.5


In [ ]:
colonnes_renommees = ["Date",	"Open",	"High",	"Low",	"Close",	"Volume",	"MFI",	"ATR",	"UpT_limit",	"DownT_support",	"Alphatrend_k1",	"Alphatrend_k2",	"Signal"]
df_ta.columns = colonnes_renommees

Absence de résultats dans df compare. OK